# News Topic Classification

The project involves a large dataset of news articles collected over several years. These articles cover a wide range of topics such as world events, sports, business, and science/technology. Each article headline is labeled with a number from 0 to 3, indicating its category, as described below. 

| Value | Topic        |
|:------|:-------------|
| 0     | World        |
| 1     | Sports       |
| 2     | Business     |
| 3     | Sci/Tech     |


Our goal is to create a model that, given an unknown article headline, can classify it into one of these 4 topics.

# Exploring Our Data Set
Our dataset consists of only two columns, *text* and *label*, as shown below:

In [5]:
import pandas as pd
df = pd.read_csv('training_data.csv')
df.head(10)

,text,label
0,Wall St. Bears Claw Back Into the Black (Reute...,2
1,Carlyle Looks Toward Commercial Aerospace (Reu...,2
2,Oil and Economy Cloud Stocks' Outlook (Reuters...,2
3,Iraq Halts Oil Exports from Main Southern Pipe...,2
4,"Oil prices soar to all-time record, posing new...",2
5,"Stocks End Up, But Near Year Lows (Reuters) Re...",2
6,Money Funds Fell in Latest Week (AP) AP - Asse...,2
7,Fed minutes show dissent over inflation (USATO...,2
8,Safety Net (Forbes.com) Forbes.com - After ear...,2
9,Wall St. Bears Claw Back Into the Black NEW Y...,2


Text is, as the named suggests, just the headline of the news article, and label has it's classification according to the table mentioned before. As it is obvious, we need to process our text into tokens in order to use it in any prediction models.

# Pre-processing and Tokenizing
We start by pre-processing our data, as described bellow, and also tokenizing it. Here we create the function **process_data()**, wich receives a dataset, as well as a number to indicate which iteration of the processing we are doing. Later in the notebook it will become clear that more processing is needed, but we didn't want to tamper with the initial results, so we created this way of adding new parts to the function, without making use of them.

### Cleaning
We start by using a regex to remove any HTML elements, to clean the text data.

### Lowercase
We then lowercase every word to ensure uniformity and prevent the model from treating words differently based on their capitalization.

### Special Characters / Punctuation
For this particular problem, special characters and punctuation marks don't contribute much to the meaning of the text and can be removed (opinion could be different if this was a sentiment analysis instead of a topic classification problem)

### Tokenization
We then split the text into individual words/tokens, as it helps capture semantic more effectively.

### Stopwords
Stopwords are common words that occur frequently in language, but, in a topic classification problem, carry virtually no useful information. Removing them reduces the noise in the data.

### Stemming
We reduce words to their base or root form, as it reduces the size of vocabulary and the dimensionality of the feature space.

In [ ]:
import pandas as pd
import re
import nltk
from nltk import PorterStemmer
from nltk.corpus import stopwords
nltk.download('stopwords')
nltk.download('punkt')

stemmer = PorterStemmer()

def stem_tokens(tokens):
    return [stemmer.stem(token) for token in tokens]

def tokenize_numbers(text):
    year_pattern = r'\b\d{4}\b'
    percentage_pattern = r'\b\d+(?:\.\d+)?%'
    time_pattern = r'\b\d+\s*(?:hours?|mins?|minutes?|secs?|seconds?|days?|weeks?|months?|years?|decades?)\b'
    number_pattern = r'\b\d+\b'
    date_pattern = r'\b\d{1,2}/\d{1,2}/\d{4}\b'  # Matches dates in mm/dd/yyyy and dd/mm/yyyy formats

    # Tokenize numbers based on patterns
    text = re.sub(percentage_pattern, 'percentagetoken', text)
    text = re.sub(date_pattern, 'datetoken', text)
    text = re.sub(year_pattern, 'yeartoken', text)
    text = re.sub(time_pattern, 'timetoken', text)
    text = re.sub(number_pattern, 'numbertoken', text)

    return text

def process_data(data, iteration):
    if(iteration >= 2):
        data['text'] = data['text'].apply(tokenize_numbers)

    # Cleaning
    data['text'] = data['text'].str.replace(r'#\d+;', '', regex=True)

    # Lowercase
    data['text'] = data['text'].str.lower()

    # Special Characters / Punctuation
    data['text'] = data['text'].str.replace(r'[^\w\s]', '', regex=True)

    # Tokenization
    data['tokens'] = data['text'].apply(nltk.word_tokenize)

    # Stopwords
    stop_words = set(stopwords.words('english'))
    if(type >= 2):
        stop_words.update(['AP', 'Reuters', 'space.com', 'techweb', 'MacCentral', 'PC World', 'USATODAY.COM', 'CNN'])
    data['tokens'] = data['tokens'].apply(lambda x: [word for word in x if word.lower() not in stop_words])

    data['tokens'] = data['tokens'].apply(stem_tokens)
    
    print(data['tokens'])
    
    return data

# Model Training
Now taht we can process our data, we can work on training some models. Here we create the function **train_data** to train a few models and compare which yield the best results when classifying the headline topics.

In [117]:
import matplotlib.pyplot as plt
import os
from sklearn.model_selection import train_test_split
from pycaret.classification import *
from sklearn.feature_extraction.text import CountVectorizer
from nltk.corpus import stopwords
from sklearn.naive_bayes import MultinomialNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score, confusion_matrix



def train_data(training_data, test_data):
    X_train = training_data['text']
    y_train = training_data['label']
    X_val = test_data['text']
    y_val = test_data['label']
    
    vectorizer = CountVectorizer(analyzer='word', max_features=5000, lowercase=True, stop_words='english', ngram_range=(1, 2))
    
    X_train = vectorizer.fit_transform(X_train)
    
    X_val = vectorizer.transform(X_val)  # Transform validation data
    
    model_nb = MultinomialNB()
    model_dt = DecisionTreeClassifier(random_state=123)
    model_rf = RandomForestClassifier(random_state=123)
    model_lr = LogisticRegression(solver='lbfgs', max_iter=1000)
        
    print('----------------------------------------------------------------')
    model_nb = model_nb.fit(X=X_train, y=y_train)
    y_pred_nb = model_nb.predict(X_val)
    print("Naive Bayes Accuracy:", accuracy_score(y_val, y_pred_nb))
    print("Naive Bayes Confusion Matrix:")
    print(confusion_matrix(y_val, y_pred_nb))
    print("Naive Bayes Classification Report:")
    print(classification_report(y_val, y_pred_nb))
    print()
    '''
    print('----------------------------------------------------------------')
    model_dt = model_dt.fit(X=X_train, y=y_train)
    y_pred_dt = model_dt.predict(X_val)
    print("Decision Tree Accuracy:", accuracy_score(y_val, y_pred_dt))
    print("Decision Tree Confusion Matrix:")
    print(confusion_matrix(y_val, y_pred_dt))
    print("Decision Tree Classification Report:")
    print(classification_report(y_val, y_pred_dt))
    print()
    
    print('----------------------------------------------------------------')
    model_rf = model_rf.fit(X=X_train, y=y_train)
    y_pred_rf = model_rf.predict(X_val)
    print("Random Forest Accuracy:", accuracy_score(y_val, y_pred_rf))
    print("Random Forest Confusion Matrix:")
    print(confusion_matrix(y_val, y_pred_rf))
    print("Random Forest Classification Report:")
    print(classification_report(y_val, y_pred_rf))
    print()
    '''
    print('----------------------------------------------------------------')
    model_lr = model_lr.fit(X=X_train, y=y_train)
    y_pred_lr = model_lr.predict(X_val)
    print("Logistic Regression Accuracy:", accuracy_score(y_val, y_pred_lr))
    print("Logistic Regression Confusion Matrix:")
    print(confusion_matrix(y_val, y_pred_lr))
    print("Logistic Regression Classification Report:")
    print(classification_report(y_val, y_pred_lr))
        

In [ ]:
training_data = pd.read_csv('training_data.csv')
#training_data = pd.read_csv('training_data_xs.csv')
test_data = pd.read_csv('test_data.csv')

training_data = process_data(training_data, 2)
test_data = process_data(test_data, 2)

In [ ]:
train_data(training_data, test_data)